**Seminario 1 - Problema 2**

Jordi Tudela

URL: https://github.com/bar0net/03MAIR---Algoritmos-de-Optimizacion/blob/master/SEMINARIO/JordiTudela-SE1-P2.ipynb


Ref. Plantilla: https://github.com/raul27868/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/SEMINARIO/Seminario(plantilla).ipynb

# Organizar los horarios de partidos de La Liga

**Posibilidades sin restricciones:** 

- 10 partidos
- 10 horarios
- En cada uno de los horarios disponibles, se pueden jugar los 10 partidos
- Posibilidades: $10^{10}$

**Posibilidades con restricciones:**

- 10 partidos
- 10 horarios
- 1 partido obligatorio en viernes: 10 posibles partidos
- 1 partido obligatorio en lunes: 9 posibles partidos restantes
- No se especifica que deba haber partidos en sábado o domingo en la definición del problema
- Cada uno de los 8 partidos restantes pueden caer en cualquier de los 10 horarios disponibles: $8^{10}$
- Posibilidades: $10*9*8^{10}$


**Estructura de datos:**

- Lista ordenada con todos los partidos de la jornada (P) representado por las 2 letras de la categoría de los dos equipos
- Lista ordenada con el horario asignado (H)

*Así, el partido con índice 0 jugará en el horario con índice 0*

Razonamiento: Es la estructura más compacta posible que nos permite definir la solución.

**Objetivo Inicial:**

$\sum_{h=0}^9 Ratio(H_h) * \frac{Juega_{h,p}*Puntos(P_p)*Audiencia(P_p)}{\sum_{p=0}^{n} Juega_{h,p}*Puntos(P_p)}$

- $H_i$ es el horario con índice 
- $P_i$ es partido con índice i
- Juega(h,p) devuelve 1 si el Partido p se juega en el Horario h, si no devuelve 0
- Puntos(P) devuelve la suma de los puntos de ambos equipos (4 por equipo A, 2 por equipo B y 1 por equipo C)


**Analisis:**

*Lemma:* Dados los parametros del problema, el óptimo evitará coincidencias horarias.

- Supongamos dos partidos: P y Q
- Supongamos que la audiencia de P ($A_p$) $\geqslant$ audiencia de Q ($A_q$)
- Así, los puntos asociados al partido P ($P_p$) $\geqslant$ los puntos asociados a Q ($P_q$)
- Supongamos dos ratios arbitrarios 0 < S < R $\leqslant$ 1
- Comparamos asignar P en horario en alta audiencia (R) y Q en baja audiencia (S) con asignar ambos en alta audiencia.

$\frac{R·P_p·A_p}{P_p}+\frac{S·P_q·A_q}{P_q} = R·A_p+S·A_q \geqslant R · \frac{P_p·max(A_p,A_q)+P_q·max(A_p,A_q)}{P_p+P_q}$

$R·A_p+S·A_q \geqslant R · \frac{P_p·A_p+P_q·A_p}{P_p+P_q} = R · A_p · \frac{P_p+P_q}{P_p+P_q} = R · A_p$

Finalmente es obvio que asignar ambos partidos en alta audiencia en lugar de en baja audiencia proporciona cifras mayores.

$R·A_p+S·A_q \geqslant R · A_p \geqslant S · A_p$

Así, se demuestra que es prioritario llenar todas las franjas horarias. Dicho de otro modo, el óptimo con 10 partidos y 10 horarios asignará 1 único partido por horario (y eso nos permite simplificar cálculos)

** Objetivo: **

$max\sum_{i=0}^{9}Ratio(H_{i})*Audiencia(P_{i})$

- $H_i$ es el horario con índice 
- $P_i$ es partido con índice i

Al tener un reparto desigual de la audiencia en horarios dónde coinciden partidos:

## Funciones de Soporte

In [1]:
#             AA   AB   AC  BA   BB    BC   CA    CB    CC
AUDIENCIA = [2.0, 1.3, 1.0, 1.3, 0.9, 0.75, 1.0, 0.75, 0.47]
RATIOS = { 'V20' : 0.4, 
           'S12' : 0.55, 'D12' : 0.45,
           'S16' : 0.7,  'D16' : 0.75,
           'S18' : 0.8,  'D18' : 0.85,
           'S20' : 1.0,  'D20' : 1.0,   'L20': 0.4}
POINTS = [4, 2, 1]

def MatchValue(x):
    i = ord(x[0]) - 65
    j = ord(x[1]) - 65
    
    return AUDIENCIA[3*i+j]

def PointValue(x):
    return POINTS[ord(x[0])-65]+POINTS[ord(x[1])-65]

def SolutionValue(P, H):
    dict_horario = {x:[] for x in RATIOS.keys()}
    for p,h in zip(P,H):
        dict_horario[h].append(p)
        
    count = 0
    for h, partidos in dict_horario.items():
        if len(partidos) == 0:
            continue
        
        count_h = 0
        total_h = 0
        max_value = 0
        for p in partidos:
            total_h += PointValue(p)
            if max_value < MatchValue(p):
                max_value = MatchValue(p)
            
        for p in partidos:
            count_h += max_value * PointValue(p)
        count += RATIOS[h] * count_h / total_h
    
    return count

In [2]:
P  = ['BC', 'BC', 'BA', 'BB', 'BB', 'CA', 'AB', 'CB', 'BA', 'BB']
ex = ['V20', 'S12', 'S16', 'S18', 'S20', 'D16', 'D16', 'D18', 'D20', 'L20']

# Test implementación con soluciones de las diapos
print(SolutionValue(P, ex))

6.515000000000001


## Fuerza Bruta

El algoritmo de fuerza bruta consiste en recorrer todas las posibilidades del espacio de posibilidades. La complejidad en el que se puede ejecutar esta exploración es $min(O(H^P), O(P^H))$.

Podemos recorrer recursivamente todos los partidos iterando en horarios (H·H·H... P veces, $O(H^P)$) o recorrer todos los horarios iterando por partidos (P·P·P... H veces, $O(P^H)$).

In [3]:
def BruteForce(P, p_assigned=[], h_assigned=[], verbose=False):
    if len(P) == 0:
        return SolutionValue(p_assigned, h_assigned), h_assigned
    
    value = 0
    best = []
    for h in RATIOS.keys():
        if verbose:
            print(h)
        new_value, new_best = BruteForce(P[1:], p_assigned + [P[0]], h_assigned + [h])
        if value < new_value:
            value, best = new_value, new_best
            
    return new_value, new_best

print("No usar, el campo de soluciónes es demasiado amplio!")

No usar, el campo de soluciónes es demasiado amplio!


## Voraz

Hemos demostrado que la solución óptima pasa por asignar 1 único partido por franja horaria. Es decir, queremos encontrar una permutación de la lista de horarios que maximice el valor de la audiencia. Esto se puede hacer de forma trivial, asignando por orden los partidos de mayor valor a los horarios de mayor audiencia.

Este algoritmo se puede ejecutar en $O(n·log(n))$, la complejidad del algoritmo sorted que utiliza Python (Una mezcla de merge sort e insertion sort que mejora en rendimiento en los mejores casos a complejidad lineal).

In [4]:
def Greedy(P):
    games = sorted(list(P), key = lambda i : PointValue(i), reverse=True)
    times = sorted(list(RATIOS.keys()), key = lambda i : RATIOS[i], reverse=True)
    
    return games, times

partidos, horarios = Greedy(P)
print(SolutionValue(partidos, horarios))

7.2425
